In [18]:
import requests
import numpy as np
import os
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
# find all images

api_url = "https://imslp.org/api.php"
page_title = 'Piano Sonata No.1, Op.4 (Chopin, Frédéric)'


params = {
    'action': 'query',
    'titles': page_title,
    'format': 'json',
    'prop': 'images'
}

response = requests.get(api_url, params=params)
data = response.json()
print(data)

page_id = list(data['query']['pages'].keys())[0]
images = data['query']['pages'][page_id]['images'] + [{'ns': 6,'title': data['query-continue']['images']['imcontinue']}]

print("-----------")
print(images)

{'query': {'pages': {'2362': {'pageid': 2362, 'ns': 0, 'title': 'Piano Sonata No.1, Op.4 (Chopin, Frédéric)', 'images': [{'ns': 6, 'title': 'File:Chopin - Piano Sonata, Op 4.pdf'}, {'ns': 6, 'title': 'File:PMLP02362-BnF btv1b525005382.pdf'}, {'ns': 6, 'title': 'File:PMLP02362-Chopin--Sonata-No1-Op4.pdf'}, {'ns': 6, 'title': 'File:PMLP02362-Chopin Klavierwerke Band 3 Peters 6208 Op 4 filter.pdf'}, {'ns': 6, 'title': 'File:PMLP02362-Chopin Klavierwerke Band 3 Peters 6208 Op 4 scan.pdf'}, {'ns': 6, 'title': 'File:PMLP02362-Chopin Op 4 Haslinger 8147 First Edition 1851.pdf'}, {'ns': 6, 'title': 'File:PMLP02362-FChopin Piano Sonata No.1, Op.4 BH13.pdf'}, {'ns': 6, 'title': 'File:PMLP02362-FChopin Piano Sonata No.1, Op.4 KullakEd.pdf'}, {'ns': 6, 'title': 'File:PMLP02362-FChopin Piano Sonata No.1, Op.4 Mikuli.pdf'}, {'ns': 6, 'title': 'File:PMLP02362-chopin-son-01.pdf'}]}}}, 'query-continue': {'images': {'imcontinue': '2362|PMLP2362-Op4-manuscript.pdf'}}}
-----------
[{'ns': 6, 'title': 'Fil

In [3]:
# Download all images

folder_path = "Trial Downloads"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    
# Download each image
for image in images:
    image_title = image['title'][5:] #5: to get rid of the File: in front of 'title'

# a generic download page for imslp images look like https://imslp.org/images/{single-digit-number}/{double-digit-number}/{file-name}.{file type}
    
    image_url = f"https://imslp.org/images/1/14/{image_title}"
    image_path = os.path.join(folder_path, image_title)

    # Download the image file
    image_response = requests.get(image_url)
    
    print(image_response)

    # Save the image to a file
    with open(image_path, 'wb') as file:
        file.write(image_response.content)
        print(f"Downloaded image {image_title}")

<Response [200]>
Downloaded image Chopin - Piano Sonata, Op 4.pdf
<Response [200]>
Downloaded image PMLP02362-BnF btv1b525005382.pdf
<Response [200]>
Downloaded image PMLP02362-Chopin--Sonata-No1-Op4.pdf
<Response [200]>
Downloaded image PMLP02362-Chopin Klavierwerke Band 3 Peters 6208 Op 4 filter.pdf
<Response [200]>
Downloaded image PMLP02362-Chopin Klavierwerke Band 3 Peters 6208 Op 4 scan.pdf
<Response [200]>
Downloaded image PMLP02362-Chopin Op 4 Haslinger 8147 First Edition 1851.pdf
<Response [200]>
Downloaded image PMLP02362-FChopin Piano Sonata No.1, Op.4 BH13.pdf
<Response [200]>
Downloaded image PMLP02362-FChopin Piano Sonata No.1, Op.4 KullakEd.pdf
<Response [200]>
Downloaded image PMLP02362-FChopin Piano Sonata No.1, Op.4 Mikuli.pdf
<Response [200]>
Downloaded image PMLP02362-chopin-son-01.pdf
<Response [200]>
Downloaded image PMLP2362-Op4-manuscript.pdf


In [4]:
# An example query using the api

info_params = {
    'action': 'query',
    'format': 'json',
    "list": "search",
    'srsearch': 'Piano Sonata No.1, Op.4 (Chopin, Frédéric)',
}

response = requests.get(api_url, params=info_params)
info_data = response.json()
print(info_data)

{'query': {'search': [{'ns': 0, 'title': 'Piano Sonata No.1, Op.4 (Chopin, Frédéric)', 'snippet': "|File Name 3=PMLP02362-<span class='searchmatch'>chopin</span>-son-01.pdf\n|Scanner 3=<span class='searchmatch'>piano</span>.ru\n", 'size': 6181, 'wordcount': 729, 'timestamp': '2023-12-28T11:17:39Z'}]}}


In [5]:
# code to get page content in HTML

url = 'https://imslp.org/wiki/Andante_spianato_et_Grande_polonaise_brillante,_Op.22_(Chopin,_Fr%C3%A9d%C3%A9ric)'
response = requests.get(url)
response.raise_for_status()
page_content = response.text
print(page_content)

<!DOCTYPE html>
<html lang="en" dir="ltr" class="client-nojs">
<head>
<title>Andante spianato et Grande polonaise brillante, Op.22 (Chopin, Frédéric) - IMSLP</title>
<meta charset="UTF-8" />
<meta name="generator" content="MediaWiki 1.18.1" />
<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1" />
<link rel="apple-touch-icon" sizes="180x180" href="/apple-touch-icon.png" />
<link rel="icon" sizes="192x192" href="/android-touch-icon.png" />
<link rel="shortcut icon" href="/favicon.ico" />
<link rel="search" type="application/opensearchdescription+xml" href="/opensearch_desc.php" title="IMSLP (en)" />
<link rel="EditURI" type="application/rsd+xml" href="//imslp.org/api.php?action=rsd" />
<link rel="copyright" href="http://creativecommons.org/licenses/by-sa/4.0/" />
<link rel="alternate" type="application/atom+xml" title="IMSLP Atom feed" href="/index.php?title=Special:RecentChanges&amp;feed=atom" />
<

In [6]:
# functions to get page text and parse out the general information sectioin

def get_page_text(url):
    # parse web content into English text
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for any request errors

        soup = BeautifulSoup(response.content, 'html.parser')
        # Extract all text from the page, excluding script and style tags
        text = soup.get_text(separator='\n', strip=True)

        return text
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

def extract_text_between_strings(text, start_string, end_string):\
    # Find the second occurrence of the start_string (the second occurence of "general information" gives us our metadata)
    start_index = text.find(start_string, text.find(start_string) + len(start_string))
    if start_index == -1:  # If start_string is not found
        return None

    # Find the occurrence of the end_string
    end_index = text.find(end_string, start_index)
    if end_index == -1:  # If end_string is not found
        return None

    # Extract the substring between the second occurrence of start_string and end_string
    extracted_text = text[start_index + len(start_string):end_index].strip()

    return extracted_text

def retrieve_general_information(url):
    # retrieve the general information section
    parsed_text = extract_text_between_strings(get_page_text(url),'General Information','Instrumentation')
    return parsed_text

In [52]:
# an example parsing

url_to_parse = ['https://imslp.org/wiki/Ballade_No.3,_Op.47_(Chopin,_Fr%C3%A9d%C3%A9ric)',
                'https://imslp.org/wiki/%C3%89tudes,_Op.10_(Chopin,_Fr%C3%A9d%C3%A9ric)',
                'https://imslp.org/wiki/Piano_Sonata_No.2%2C_Op.36_(Rachmaninoff%2C_Sergei)',
                'https://imslp.org/wiki/Barcarolle_No.11%2C_Op.105_(Faur%C3%A9%2C_Gabriel)',
                'https://imslp.org/wiki/English_Suite_No.1_in_A_major%2C_BWV_806_(Bach%2C_Johann_Sebastian)']

def get_attribute_from_single_url(url):
    parsed_text = retrieve_general_information(url)
    
    if parsed_text:
        def extract_text_between_strings_new(text, start_string, end_string):
            # Find the second occurrence of the start_string (the second occurence of "general information" gives us our metadata)
            start_index = text.find(start_string)
            if start_index == -1:  # If start_string is not found
                return None

            # Find the occurrence of the end_string
            end_index = text.find(end_string, start_index)
            if end_index == -1:  # If end_string is not found
                return None

            # Extract the substring between the second occurrence of start_string and end_string
            extracted_text = text[start_index + len(start_string):end_index].strip()

            return extracted_text

        info = {}
        attribute_list = ['Work Title', 'Alt\nernative\n.\nTitle', 'Name Translations', 'Name Aliases', 'Authorities', 'Composer',
                        'Opus/Catalogue Number', 'I-Catalogue Number', 'Key', 'Movements/Sections', 'Year/Date of Composition', 
                        'First Pub\nlication', 'Dedication', 'Average Duration', 'Composer Time Period', "Piece Style"]
        
        present_attributes = []
        # check which attributes are present in the parsed text
        for attribute in attribute_list:
            if parsed_text.find(attribute) != -1:
                present_attributes.append(attribute)
        
        # extract the text for each present attribute
        for attribute in attribute_list:
            if attribute in present_attributes:
                info[attribute] = extract_text_between_strings_new(parsed_text, attribute, present_attributes[present_attributes.index(attribute) + 1] if present_attributes.index(attribute) != len(present_attributes) - 1 else "Romantic")
                
            else:
                info[attribute] = None
                
        info['Piece Style'] = parsed_text[parsed_text.find('Piece Style')+12:] # extract the piece style from the end of the general information section
        info['url'] = url
        
        return info
        
    else:
        print("String not found or invalid input.")
        return None

info = []
for url in url_to_parse:
    info.append(get_attribute_from_single_url(url))
        
# convert the dictionary to a pandas dataframe
df_info = pd.DataFrame(info)
df_info.rename(columns={'Alt\nernative\n.\nTitle': 'Alternative Title'}, inplace=True) # rename the column to remove the new line characters
df_info.rename(columns={'First Pub\nlication': 'First Publication'}, inplace=True)
print(df_info.head())

           Work Title Alternative Title  \
0        Ballade No.3                     
1              Etudes                     
2   Piano Sonata No.2                     
3    Barcarolle No.11                     
4  English Suite No.1                     

                                   Name Translations  \
0  Ballade No. 3 in A-flat major, Op. 47\n;\nBall...   
1  Étude Op. 10, No. 10\n;\n연습곡 10번\n;\nEtiuda op...   
2  第2号钢琴奏鸣曲\n;\n第2號鋼琴奏鳴曲\n;\nピアノソナタ第2番\n;\nKlavir...   
3                                               None   
4                                               None   

                                        Name Aliases  \
0                            Balada n.º 3\n;\nOp. 47   
1                                               None   
2  降B小調第二號鋼琴奏鳴曲\n;\nSergej Rahmaninov: Klavirska ...   
3                                               None   
4                                               None   

                                         Authorities               